In [2]:
# リスト 2.1.1 
# 青空文庫（Aozora Bunko）から夏目漱石『三四郎』の本文テキストを取得する最小例
# -----------------------------------------------------------------------------
# 目的:
#   - 公開されている zip アーカイブ（Shift_JIS のテキストを含む）をダウンロード
#   - zip を展開し、中のテキストファイルを読み出して文字列変数 `text_org` に格納
#
# ポイント:
#   - Aozora のテキストは多くが Shift_JIS（sjis, cp932 互換）で配布される
#   - `urllib.request.urlretrieve` を用いた単純ダウンロード（進捗/例外処理は最小）
#   - `zipfile.ZipFile.extractall()` は作業ディレクトリ直下に展開する
#   - zip 内に複数ファイルがある場合、このコードでは「最後に読んだファイル」で text_org が上書きされる
#     （『三四郎』の配布物は通常 1 本の .txt なので実運用上は問題になりにくい）
#   - 変数名 `zip` は Python 組み込み関数 zip と同名だが、このスニペットでは影響はない
#     （大規模スクリプトでは衝突回避のため別名推奨）
# -----------------------------------------------------------------------------

# zipファイルのダウンロード元URL（ルビ付きテキスト配布物）
url = 'https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip'

# ローカルへ保存する zip ファイル名（作業ディレクトリ配下に保存）
zip = '794_ruby_4237.zip'

# ネットワーク経由でファイルを取得するための標準ライブラリ
import urllib.request

# URL からローカルファイルへ直接保存するユーティリティ関数
# - 戻り値は (保存先パス, ヘッダ) だが、ここでは副作用（保存）のみを利用
urllib.request.urlretrieve(url, zip)

# ダウンロードした zip ファイルの解凍に用いる標準ライブラリ
import zipfile

# 読み取り専用モードで zip を開く。コンテキストマネージャで安全にクローズされる
with zipfile.ZipFile(zip, 'r') as myzip:
    # zip 内の全ファイルを現在の作業ディレクトリ直下に展開
    # 注意: extractall はパス情報を信用して展開するため、未知のアーカイブでは Zip Slip 対策が必要
    #      （Aozora の公式配布物は信頼できる前提でそのまま展開）
    myzip.extractall()

    # 展開対象となった zip 内の各エントリ（ファイル情報）にアクセス
    for myfile in myzip.infolist():
        # 展開後のファイル名（相対パス）。多くの場合は .txt が 1 本
        filename = myfile.filename

        # Aozora のテキストは Shift_JIS（sjis）で配布されることが多い
        # - Windows 系の cp932 と互換性がある
        # - もしデコードエラーが発生する場合は errors='ignore' や 'replace' を検討
        # - 改行は CRLF の場合があり、解析前に正規化（\r\n -> \n）したいときは追加処理を入れる
        with open(filename, encoding='sjis') as file:
            # ファイル全体を文字列として読み込む
            text_org = file.read()

            # 参考: Aozora 独自のルビ・傍点表記が含まれることがある
            #  - 例: 《ルビ》, ｜縦棒, ［＃…］注記 など
            #  - NLP 前処理ではこれらの除去・正規化を行うケースが多い

In [3]:
# リスト 2.1.2
# テキスト内容の確認（前処理・正規化の前に、取得した生テキストの先頭/末尾を目視確認する）
# --------------------------------------------------------------------------------
# 目的:
#   - `text_org` の先頭600文字と末尾300文字を表示し、文字コード/改行/注記/ルビなどの
#     クリーニング要否を判断するための「粗いスナップショット」を得る。
# 注意:
#   - `text_org` は前段のスニペット（リスト2.1.1）で定義されている想定。
#   - 文字数が不足していても Python のスライスは安全（範囲外でも例外は出ない）。
#   - 端末/環境によっては Shift_JIS 由来文字の表示が化けることがある（IDE のフォント設定等を確認）。
# --------------------------------------------------------------------------------

# 先頭部分の見出し
print("【整形前文頭部分】")

# 先頭600文字をそのまま表示
# - Aozora のテキストには《ルビ》や［＃…］注記が混在することが多い
# - 行頭の全角スペースやCRLF(\r\n)などの改行もそのまま確認できる
print(text_org[:600])

# 区切り（可視的なセパレータ）。読みやすさのために空行＋アスタリスクの帯を挿入
print()
print("*" * 100)
print()

# 末尾部分の見出し
print("【整形前文末部分】")

# 末尾300文字を表示
# - 末尾には「底本情報」「入力者・校正者」「作成日」などのメタ情報が付くことが多い
# - 解析前に削除・分離するかの判断材料として目視する
print(text_org[-300:])

# 最後にもう一度区切り線
print("*" * 100)
print()

【整形前文頭部分】
三四郎
夏目漱石

-------------------------------------------------------
【テキスト中に現れる記号について】

《》：ルビ
（例）頓狂《とんきょう》

｜：ルビの付く文字列の始まりを特定する記号
（例）福岡県｜京都郡《みやこぐん》

［＃］：入力者注　主に外字の説明や、傍点の位置の指定
　　　（数字は、JIS X 0213の面区点番号またはUnicode、底本のページと行数）
（例）※［＃「魚＋師のつくり」、第4水準2-93-37］

〔〕：アクセント分解された欧文をかこむ
（例）〔ve'rite'《ヴェリテ》 vraie《ヴレイ》.〕
アクセント分解についての詳細は下記URLを参照してください
http://www.aozora.gr.jp/accent_separation.html
-------------------------------------------------------

［＃７字下げ］一［＃「一」は中見出し］

　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂《とんきょう》な声を出して駆け込んで来て、いきなり肌《はだ》をぬいだと思ったら背中にお灸《きゅう》のあとがいっぱいあったので、三四郎《さん

****************************************************************************************************

【整形前文末部分】
も答えなかった。ただ口の中で迷羊《ストレイ・シープ》、迷羊《ストレイ・シープ》と繰り返した。



底本：「三四郎」角川文庫クラシックス、角川書店
　　　1951（昭和26）年10月20日初版発行
　　　1997（平成9）年6月10日127刷
初出：「朝日新聞」
　　　1908（明治41）年9月1日〜12月29日
入力：古村充
校正：かとうかおり
2000年7月1日公開
2014年6月19日修正
青空文庫作成ファイル：
このファイルは、インターネットの図書館、青空文庫（http://www.aozora.gr.jp/）で作られました。入力、校正、

In [4]:
# リスト 2.1.3
# テキストの整形（青空文庫の本文からメタ記法・注記などを取り除き、解析しやすい一次テキストへ写像する）
# ------------------------------------------------------------------------------------------------
# 目的:
#   - 取得済みの生テキスト `text_org` に対し、本文以外の領域（ヘッダ/フッタ）やメタ記法（ルビ・注記）を除去する。
# 背景:
#   - 青空文庫のテキストは一定の規約（区切りの罫線、底本情報、ルビや注記の記法）に基づく。
#   - 本スニペットはその規約に依存した「単純規則ベース」の変換であり、厳密な正規化器ではない。
# 注意:
#   - 本コードは既存処理の意味を変えない範囲でコメントのみを詳細化している（正規表現の改善提案は記述のみ）。
#   - 形式が異なる作品や改版では想定外の挙動（IndexError など）があり得るため、実運用ではバリデーションが必要。

import re  # 正規表現によるパターン置換/分割を用いる

# ヘッダ部分の除去
# --------------------------------------------------------------------------------
# 青空文庫の多くの作品では、冒頭にメタ情報（作品名、著者、配布条件など）があり、
# その後に「-----」のような連続ハイフン罫線が区切りとして現れることがある。
# `re.split('\-{5,}', text_org)` は「ハイフンが5回以上連続する箇所」で分割する。
# 典型的には [ヘッダ, （中間情報）, 本文, …] のように複数セグメントに分かれる想定で、
# ここではインデックス [2]（3番目のセグメント）を本文とみなして取得している。
# リスク:
#   - 作品により区切りの本数・位置が異なる場合があり、[2] が存在しないと IndexError。
#   - 「-----」が本文中に出現すれば誤分割の可能性。
# 対策（提案）:
#   - 例外処理でフォールバック、あるいはタイトル行や底本開始トリガと併用して決定。
text = re.split("\-{5,}", text_org)[2]

# フッタ部分の除去
# --------------------------------------------------------------------------------
# 底本情報は通常「底本：」という固定フレーズで始まるため、そこから後ろを落とす。
# `re.split('底本：', text)` により、先頭要素 [0] が本文側、[1] 以降が底本情報と想定される。
# リスク:
#   - 「底本：」が存在しない版では分割されず、全文が本文扱いになる（この挙動は安全側）。
#   - 本文中に「底本：」が出るケースは稀だが、理論上は誤切断の可能性あり。
text = re.split("底本：", text)[0]

# | の除去
# --------------------------------------------------------------------------------
# 青空文庫のルビ記法では、ルビ対象の語の前に「｜」（全角縦棒 U+FF5C）を置く。
# ここでは ASCII の '|'（U+007C）を除去している点に注意。全角の「｜」は削除対象にならない。
# 実際の原文が全角を用いている場合、本行の効果は限定的。
# 対策（提案）:
#   - 必要に応じて text.replace('｜', '') を追加し、全角縦棒にも対応する。
text = text.replace("|", "")

# ルビの削除
# --------------------------------------------------------------------------------
# 青空文庫ではルビは《…》で囲われる（例：「彼《かれ》」）。
# `re.sub('《.+?》', '', text)` は最短一致（?）で《 と 》に挟まれた最小区間を空文字へ置換する。
# 特性:
#   - デフォルトでは '.' は改行にマッチしないため、改行をまたぐ異常なルビブロックには影響しない。
#   - ネストや《…》の不整合があると想定外の残存が起き得るが、通常の青空文庫テキストでは問題になりにくい。
# 代替案（提案）:
#   - 品質を上げるなら、改行を含まないクラス `[^ \n]` を使う、もしくは事前のバリデーションを行う。
text = re.sub("《.+?》", "", text)

# 入力注の削除
# --------------------------------------------------------------------------------
# 入力者注・傍注は多くの場合「［＃…］」形式の角括弧付き注記として表現される。
# `re.sub('［＃.+?］', '', text)` はこれらを最短一致で除去する。
# 注意:
#   - こちらも '.' は改行非貪欲。複数行にまたがる注記にはマッチしない。
#   - テキスト内に同様の記号が本文用途で現れた場合、誤消去のリスクがある（稀）。
text = re.sub("［＃.+?］", "", text)

# 空行の削除・改行規範の統一
# --------------------------------------------------------------------------------
# `re.sub('\n\n', '\n', text)` は「連続する1組の空行」を1回だけ詰める。
# つまり、3行以上の連続空行がある場合、1回の適用では空行がまだ残る（例: '\n\n\n' -> '\n\n'）。
# より積極的に詰めたい場合は '\n{2,}' を 1 回で '\n' に置換するのが一般的。
text = re.sub("\n\n", "\n", text)

# Windows 由来の CRLF を LF に正規化するため '\r' を除去。
# これにより改行は '\n' に統一され、後段のトークナイザや行単位処理の安定性が向上する。
text = re.sub("\r", "", text)

In [5]:
# リスト 2.1.4
# 整形結果の確認

# 頭の100文字の表示
print("【整形後文頭部分】")
print(text[:100])

# 区切り表示
print()
print("*" * 100)
print()

# 後ろの100文字の表示
print("【整形後文末部分】")
print(text[-100:])

【整形後文頭部分】

一
　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬい

****************************************************************************************************

【整形後文末部分】
評に取りかかる。与次郎だけが三四郎のそばへ来た。
「どうだ森の女は」
「森の女という題が悪い」
「じゃ、なんとすればよいんだ」
　三四郎はなんとも答えなかった。ただ口の中で迷羊、迷羊と繰り返した。


